In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/notebooks/")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
confini = gpd.read_file(data_folder+'clean_data/confini_piemonte/confini_piemonte.shp')
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2018.csv', encoding='utf-16', sep='\t')

In [ ]:
from shapely.geometry import Point
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)

gdf = gdf.set_crs(confini.crs)

def search_polygon(confini, point):
    if len(confini[point.within(confini)]) > 0:
        return confini[point.within(confini)].iloc[0]
    else:
        return None

ifc_2018 = gdf.copy()
ifc_2018['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))
ifc_2018.head()

In [ ]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2019.csv', encoding='utf-16', sep='\t')
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_crs(confini.crs)

ifc_2019 = gdf.copy()
ifc_2019['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))

df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2021.csv', encoding='utf-16', sep='\t')
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_crs(confini.crs)

ifc_2021 = gdf.copy()
ifc_2021['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))


In [ ]:
lst = [ifc_2018, ifc_2019, ifc_2021]
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2018_valore.csv', encoding='utf-16', sep='\t')
lst[0].loc[688, 'Comune'] = 'None Torinese'
df2018 = df.merge(lst[0], on='Comune', how='inner')

In [ ]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2019_valore.csv', encoding='utf-16', sep='\t')
df2019 = df.merge(lst[1], on='Comune', how='inner')

df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2021_valore.csv', encoding='utf-16', sep='\t')
df[df.Comune.isna()]['Comune'] = 'None Torinese'
df2021 = df.merge(lst[2], on='Comune', how='inner')

In [ ]:
lstdf = [df2018, df2019, df2021]

# for df in lstdf:
#     df.drop(columns = df.columns[[1, 2, 3, 4, 5, 6, 20, 21, 22, 23, 28, 29, 30, 31, 32, 33]], inplace=True)
#     df = gpd.GeoDataFrame(df)

In [ ]:
lstdf[0].loc[164, lstdf[0].columns[-5:]]

# for df in lstdf:
#     df.loc[164, df.columns[-5:-1]] = [2.1, 'Medio-bassa', '5.001-10.000', 'Piccole città e sobborghi']
#     df.loc[164, 'geometry'] = confini[confini['Codice Com'] == 'A001168'].geometry.iloc[0]

In [ ]:
for n, df in zip([2018, 2019, 2021], lstdf):
    gpd.GeoDataFrame(df).to_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/IFC/{n}')

---